In [39]:
def euclidean_distance(point1, point2): 
    # written in a different way from the one we used in 1-NN.ipynb
    sum_sq = 0.0
    for i in range(len(point1)):
        sum_sq += (point1[i] - point2[i]) ** 2
    return sum_sq ** 0.5

def classify(dataset, target_point, k):
    if k <= 0:
        raise ValueError("k must be a positive integer")
    if not dataset:
        return None

    # Compute distances and store with labels
    distances = []
    for data_point in dataset:
        dist = euclidean_distance(target_point, data_point[0])
        distances.append((dist, data_point[1]))
    
    # Sort by distance (ascending)
    distances.sort(key=lambda x: x[0])
    
    #for debugging
    print("Colors nearby with distances (sorted nearest to farthest): \n ", distances,"\n")
    
    # Extract first k neighbors
    k_nearest = distances[:k]
    
    #for debugging
    print("The nearest ", k, ": ", k_nearest, "\n")
    
    # Count label frequencies
    label_counts = {}
    for _, label in k_nearest:
        label_counts[label] = label_counts.get(label, 0) + 1
        
    #for debugging
    print("Color counts in the nearest", k, "neighbors: " , label_counts, "\n")
    
    # Find max frequency and candidate labels
    max_count = max(label_counts.values())
    candidates = [label for label, count in label_counts.items() if count == max_count]
    

    
    # Return the majority label if no tie
    if len(candidates) == 1:
        return candidates[0]
    
    # Tie-breaking: pick the candidate with the closest occurrence
    for dist, label in k_nearest:
        if label in candidates:
            return label
        


In [40]:
import numpy as np

data = [[[255,0,0],"RED"], [[255,51,0],"RED"], [[255,102,0],"RED"], [[204,0,0],"RED"], [[190,0,20],"RED"], [[0,0,153],"BLUE"], [[0,51,204],"BLUE"], [[51,51,255],"BLUE"], [[0,102,255], "BLUE"], [[204,0,198],"BLUE"]]
target = np.array([255,51,120])

In [41]:
classify(data, target,3)

Colors nearby with distances (sorted nearest to farthest): 
  [(106.23558725775464, 'BLUE'), (120.0, 'RED'), (129.71507237017602, 'RED'), (130.3878828726044, 'RED'), (130.3878828726044, 'RED'), (140.0071426749364, 'RED'), (244.62420158275427, 'BLUE'), (262.1354611646429, 'BLUE'), (268.47904946196456, 'BLUE'), (293.00341294940574, 'BLUE')] 

The nearest  3 :  [(106.23558725775464, 'BLUE'), (120.0, 'RED'), (129.71507237017602, 'RED')] 

Color counts in the nearest 3 neighbors:  {'BLUE': 1, 'RED': 2} 



'RED'

Here's an optimized version using NumPy for vectorized operations, which significantly improves performance. We are not using the big sklearn yet, all this is just for learning

Key Optimizations:

    Vectorized Distance Calculation
    Uses np.linalg.norm for efficient Euclidean distance computation across all points simultaneously.

    Partial Sorting
    Uses np.argpartition (O(n) time) instead of full sorting (O(n log n)) to find k-nearest neighbors.

    NumPy-based Counting
    Leverages np.unique with counts for efficient label frequency calculation.

    Tie-breaking
    Among majority candidates, selects the label with the smallest distance using the original neighbor ordering.

Performance Comparison:

Distance Calculation: 
Before: O(n) with Python loops, but After: O(1) vectorized

Neighbor Selection:
Before: O(n log n) sorting,  but After: O(n) partitioning

Memory Usage here is Moderate (2 arrays)

Before is best for Small datasets (n < 1k) while After is for Large datasets

In [ ]:
import numpy as np

def classify(dataset, target_point, k=1):
    if k <= 0:
        raise ValueError("k must be a positive integer")
    if not dataset:
        return None

    # Convert to NumPy arrays
    data_points = np.array([item[0] for item in dataset])
    labels = np.array([item[1] for item in dataset])
    target = np.array(target_point)
    
    # Vectorized distance calculation
    distances = np.linalg.norm(data_points - target, axis=1)
    
    # Get k-nearest neighbors
    nearest_indices = np.argpartition(distances, k)[:k]
    k_nearest_labels = labels[nearest_indices]
    
    # Count label frequencies
    unique_labels, counts = np.unique(k_nearest_labels, return_counts=True)
    max_count = np.max(counts)
    
    # Handle ties by selecting the closest label
    if np.sum(counts == max_count) == 1:
        return unique_labels[np.argmax(counts)]
    else:
        # Among tied labels, pick the one with minimal distance
        candidate_labels = unique_labels[counts == max_count]
        candidate_mask = np.isin(k_nearest_labels, candidate_labels)
        closest_candidate_index = nearest_indices[candidate_mask][0]
        return labels[closest_candidate_index]